# Introduction to TensorFlow Core APIs
This a notebook following the guide https://www.tensorflow.org/guide/low_level_intro

Recall that
* computations in TensorFlows happen by executing a ``tf.Graph``,
* the graph can be defined but not necessarily run,
* run is performed via a ``tf.Session`` object.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

**Definition** (*Tensor*) a *tensor* (more precisely *tensor value*) in mathematical sense is a $N$ dimensional vector into some field. For example, in the field of real numbers $\mathbb{R}$, $\mathbf{x} = [0,-1,\pi, 0.99]$ is a tensor. Notice that vectors and matrices are special case of tensor with $1$ and $2$ dimensions, respectively. The number of dimension is called the *rank* of the tensor and the list of the number of elements for in each dimension is called the *shape*. In the previous example the rank is $1$ (*i.e.*, vector) and the shape is $[4]$.

Notice that the concept of tensor is very well represented by ``numpy`` ``ndarray`` objects, which is, in fact, what TensorFlow uses under the hood.

In [2]:
t = np.asarray([0,-1, np.pi, 0.99])

## Building the graph

As stated above the first part in a TensorFlow computation is the construction of a graph which is an object ``tf.Graph``. In TensorFlow nodes and edges of the graph are object of type ``tf.Operation`` and ``tf.Tensor``, respectively. Notice that ``tf.Tensor`` is not a tensor value rather a "placeholder" for what can be substituted with a tensor value.

It is interesting to notice how the documentation defines the ``tf.Tensor`` object as representing the "ouput of an ``Operation``". This stresses the fact that ``tf.Tensor`` objects are to be interpreted as ouputs and therefore they should always be constructed as result from some operation (indeed an edge of a graph must always have endpoints, constructing a ``Tensor`` from an ``Operation`` - *i.e.*, a node - guarantees that the starting end is always defined).

In [3]:
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
d = tf.constant([[1.0, 1.0], [0.0, 1.0]])
e = tf.matmul(c, d)
sess = tf.compat.v1.Session()
result = sess.run(e)
print(result)

[[1. 3.]
 [3. 7.]]


The result is what is to be expected, however to obtain the actual values of the matrix ``e=c*d`` we had to run the graph in a session.

In [4]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)
total = a + b
print(a)
print(b)
print(total)

Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("Const_3:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


Again ``Tensor`` do not have actal values, to produce result we need to run the computation (run the graph)

In [5]:
with tf.Session() as sess:
    result = sess.run(total)
    print(result)

7.0


In [6]:
delta = tf.abs(a - b)
with tf.Session() as sess:
    print(sess.run({"Tot." : total, "Delta" : delta}))

{'Tot.': 7.0, 'Delta': 1.0}


Because ``Tensor`` do not contain values, we can initialize them to be just "placeholder" for values that will be obtained during computation itself

In [7]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

When an ``Session`` is executed the placeholders can be filled passing a the ``feed_dict`` dictionary to the ``run`` method of the ``Session``

In [8]:
with tf.Session() as sess:
    print(sess.run(z, feed_dict={x : "-3", y : "0.01"}))
    print(sess.run(z, feed_dict={x : [1,3], y : [4,5]}))

-2.99
[5. 8.]


There are more preferable ways to feed data into a graph during runs, in particular the ``tf.data.Dataset`` object serves this purpse, but to use it a proper iterator is needed

In [9]:
data = [
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7]
]

In [10]:
slices = tf.data.Dataset.from_tensor_slices(data)
next_item = slices.make_one_shot_iterator().get_next()

W0703 16:21:08.468636 140200443148096 deprecation.py:323] From <ipython-input-10-3d2b422be644>:2: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


Even following step-by-step the official guide, you get a deprecated warning (sic!)

In [11]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_item))
        except tf.errors.OutOfRangeError:
            break

[0 1]
[2 3]
[4 5]
[6 7]


## Layers

Constructing graphs defining each single node may be tedious, moreover there could be the opportunity to re-use entire subgraph in different places. Besides higher level APIs (*e.g.* ``tf.keras``), even at low level TensorFlow uses the notion of *layer*. Many ready-to-use layers are available in TensorFlow, most notably the ``td.layers.Dense`` which constitutes a basic building block for MLP.

In [12]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

W0703 16:21:08.626101 140200443148096 deprecation.py:506] From /home/skimmy/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


The ``Dense`` object has many interesting property

In [13]:
print({
    "units"      : linear_model.units,
    "activation" : linear_model.activation,
    "trainable"  : linear_model.trainable,
    "use_bias"   : linear_model.use_bias,
    "kernel_constraint" : linear_model.kernel_constraint,
})

{'units': 1, 'activation': <function linear at 0x7f82b336c8c8>, 'trainable': True, 'use_bias': True, 'kernel_constraint': None}


layers need some initialization, and we can use a default initializer

In [14]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(y, {x : [[1,2,3],[4,5,6]]}))

[[-2.7126412]
 [-5.5331855]]


Finally let's try to train a linear model

Define the data

In [15]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

Define and initialize the model

In [16]:
linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

make some untrained predictions

In [17]:
print(sess.run(y_pred))

[[-0.3049283]
 [-0.6098566]
 [-0.9147849]
 [-1.2197132]]


In [18]:
# loss function
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)
print(sess.run(loss))

W0703 16:21:09.489116 140200443148096 deprecation.py:323] From /home/skimmy/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


1.1480765


In [19]:
# Optimization
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [20]:
for i in range(100):
    _, loss_value = sess.run((train,loss))
    print(loss_value)

1.1480765
0.8581888
0.6566738
0.51648104
0.4188404
0.3507281
0.3031072
0.26970682
0.24617584
0.22949536
0.2175703
0.20894697
0.20261683
0.19787979
0.19425038
0.1913916
0.18906945
0.18712178
0.18543594
0.18393376
0.18256103
0.18128008
0.18006474
0.178897
0.17776409
0.17665736
0.17557067
0.17449978
0.17344186
0.1723947
0.17135689
0.17032748
0.1693057
0.16829109
0.16728327
0.16628197
0.16528705
0.16429824
0.16331553
0.16233885
0.16136806
0.16040316
0.15944403
0.15849066
0.15754306
0.15660107
0.1556648
0.15473408
0.15380895
0.15288934
0.15197524
0.15106657
0.15016338
0.14926562
0.14837313
0.14748603
0.14660423
0.14572772
0.14485642
0.14399034
0.1431295
0.14227371
0.1414231
0.14057757
0.13973707
0.13890159
0.13807113
0.13724563
0.13642506
0.1356094
0.13479862
0.13399266
0.13319157
0.13239518
0.13160363
0.13081682
0.13003466
0.12925719
0.12848443
0.12771624
0.12695265
0.12619358
0.12543909
0.124689125
0.123943605
0.12320258
0.12246597
0.12173375
0.12100595
0.12028248
0.11956333
0.11884846
0.

In [21]:
# prediction
print(sess.run(y_pred))

[[-0.54183435]
 [-1.2625558 ]
 [-1.9832773 ]
 [-2.7039988 ]]


At the end it is always a good idea to close the session (in fact the best way is to use ``with``)

In [22]:
sess.close()